# Imports

In [50]:
import time 
import tracemalloc

# Dataset

In [51]:
dataset_10 = [3, 8, 5, 11, 9, 4, 7, 2, 6, 1]
print(f"dataset_10")
print(f"size: {len(dataset_10)}")
print(f"total values: {sum(dataset_10)}\n")

dataset_40 = [33, 3, 23, 10, 31, 38, 4, 24, 16, 29, 27, 28, 1, 11, 40, 19, 32, 9, 26, 20, 36, 30, 8, 34, 21, 35, 13, 18, 37, 7, 5, 15, 12, 25, 2, 17, 22, 14, 39, 6]
print(f"dataset_40")
print(f"size: {len(dataset_40)}")
print(f"total values: {sum(dataset_40)}\n")

dataset_80 = [76, 62, 22, 23, 56, 27, 26, 51, 59, 57, 20, 44, 29, 80, 1, 68, 63, 34, 19, 39, 4, 53, 46, 2, 70, 43, 17, 36, 5, 60, 7, 40, 24, 6, 49, 42, 54, 11, 69, 3, 78, 79, 33, 25, 10, 18, 61, 13, 38, 47, 9, 35, 55, 66, 74, 12, 8, 50, 58, 41, 31, 14, 16, 15, 37, 72, 65, 73, 45, 75, 67, 30, 32, 28, 21, 71, 48, 77, 52, 64]
print(f"dataset_80")
print(f"size: {len(dataset_80)}")
print(f"total values: {sum(dataset_80)}\n")

dataset_10
size: 10
total values: 56

dataset_40
size: 40
total values: 820

dataset_80
size: 80
total values: 3240



# Dynamic Programming

In [52]:
def findPartition(arr, n):
	# Calculate the sum of all elements
	total_value = sum(arr)

	if total_value % 2 != 0:
		return False
	
	desired_total_value = total_value // 2

	part_table = [[True for i in range(n + 1)] for j in range(desired_total_value + 1)]

	# Initialize top row as true
	for i in range(0, n + 1):
		part_table[0][i] = True

	# Initialize leftmost column, except part_table[0][0], as false
	for i in range(1, desired_total_value + 1):
		part_table[i][0] = False

	# Fill the partition table in bottom up manner
	for i in range(1, desired_total_value + 1):

		for j in range(1, n + 1):
			part_table[i][j] = part_table[i][j - 1]

			if i >= arr[j - 1]:
				part_table[i][j] = (part_table[i][j] or
							part_table[i - arr[j - 1]][j - 1])

	return part_table[desired_total_value][n]

In [53]:
def dp_experiment(arr):
    n = len(arr)

    start_time = time.time()
    tracemalloc.start()

    result = findPartition(arr, n)

    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    print(f"Using {n} elements:")
    print(f"Time needed: {(end_time - start_time) * 1000} ms")
    print(f"Highest memory usage: {peak} bytes")

    if result == True:
        print("Can be divided into two subsets of equal sum")
    else:
        print("Can not be divided into two subsets of equal sum")

In [107]:
dp_experiment(dataset_10)

Using 10 elements:
Time needed: 0.9813308715820312 ms
Highest memory usage: 4376 bytes
Can be divided into two subsets of equal sum


In [47]:
dp_experiment(dataset_40)

Using 40 elements:
Time needed: 28.996706008911133 ms
Highest memory usage: 194280 bytes
Can be divided into two subsets of equal sum


In [49]:
dp_experiment(dataset_80)

Using 80 elements:
Time needed: 236.94586753845215 ms
Highest memory usage: 1295216 bytes
Can be divided into two subsets of equal sum


# Branch and Bound

In [60]:
def partition_values_from_index(values, start_index, total_value, unassigned_value, test_assignment, test_value, best_assignment, best_err):
    # If start_index is beyond the end of the array, then all entries have been assigned.
    if start_index >= len(values):
        # We're done. See if this assignment is better than what we have so far.
        test_err = abs(2 * test_value - total_value)
        if test_err < best_err[0]:
            # This is an improvement. Save it.
            best_err[0] = test_err
            best_assignment[:] = test_assignment[:]
            
    else:
        # See if there's any way we can assign the remaining items to improve the solution.
        test_err = abs(2 * test_value - total_value)
        if test_err - unassigned_value < best_err[0]:
            # There's a chance we can make an improvement.
            # We will now assign the next item.
            unassigned_value -= values[start_index]

            # Try adding values[start_index] to set 1.
            test_assignment[start_index] = True
            partition_values_from_index(values, start_index + 1, 
                                        total_value, unassigned_value, 
                                        test_assignment, test_value + values[start_index], 
                                        best_assignment, best_err)

            # Try adding values[start_index] to set 2.
            test_assignment[start_index] = False
            partition_values_from_index(values, start_index + 1, 
                                        total_value, unassigned_value, 
                                        test_assignment, test_value, 
                                        best_assignment, best_err)

In [61]:
def bnb_experiment(values):
    start_index = 0
    total_value = sum(values)
    unassigned_value = total_value
    test_assignment = [False] * len(values)
    test_value = 0
    best_assignment = [False] * len(values)
    best_err = [float('inf')]

    start_time = time.time()
    tracemalloc.start()

    partition_values_from_index(values, start_index, total_value, unassigned_value, test_assignment, test_value, best_assignment, best_err)

    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    print(f"Using {len(values)} elements:")
    print(f"Best error: {best_err[0]}")
    print(f"Best assignment: {best_assignment}")
    print(f"Time needed: {(end_time - start_time) * 1000} ms")
    print(f"Highest memory usage: {peak} bytes")

In [180]:
bnb_experiment(dataset_10)

Using 10 elements:
Best error: 0
Best assignment: [True, True, True, True, False, False, False, False, False, True]
Time needed: 2.00653076171875 ms
Highest memory usage: 160 bytes


Untuk dataset_40 dan dataset_80, waktu yang dibutuhkan sangat lama, hingga lebih dari 100 menit masih belum selesai.

In [ ]:
bnb_experiment(dataset_40)

In [ ]:
bnb_experiment(dataset_80)